<a href="https://colab.research.google.com/github/Mikimorka/Test/blob/main/predpove%C4%8F_ceny_bytu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# import nltk # Ak ešte nemáš, odkomentuj a spusti: nltk.download('stopwords')

# 1. Načítanie dát
# Predpokladajme, že váš CSV súbor má stĺpce 'popis_nehnutelnosti' a 'cena'
try:
    data = pd.read_csv('reality_data.csv')
    print("Dáta úspešne načítané. Prvých 5 riadkov:")
    print(data.head())
    print("\nInformácie o dátach:")
    data.info()
except FileNotFoundError:
    print("Súbor 'reality_data.csv' sa nenašiel. Prosím, uistite sa, že súbor existuje v rovnakom adresári.")
    print("Pre spustenie príkladu vytvoríme fiktívne dáta.")
    # Vytvorenie fiktívnych dát pre ukážku, ak súbor neexistuje
    data = pd.DataFrame({
        'popis_nehnutelnosti': [
            "Priestranný 3-izbový byt v centre mesta, blízko MHD a obchodov. Kompletná rekonštrukcia, balkón, tiché prostredie.",
            "Malý 1-izbový byt na okraji mesta, potrebuje rekonštrukciu. Ideálny pre študenta alebo investíciu.",
            "Veľký rodinný dom s garážou a záhradou na dedine. Kľudné prostredie, vhodný pre rodiny s deťmi.",
            "Moderný 2-izbový byt v novostavbe s výhľadom na hrad. Energetický certifikát A. Parkovacie miesto.",
            "Starší 4-izbový dom v pôvodnom stave, veľký pozemok. Vhodný na rozsiahlu rekonštrukciu alebo demoláciu.",
            "Slnečný 3-izbový byt v tichej lokalite, výborná dostupnosť do centra. Bezproblémové parkovanie.",
            "Luxusný 5-izbový mezonet s terasou a bazénom na streche. Výhľad na celé mesto. Klimatizácia."
        ],
        'cena': [250000, 120000, 350000, 280000, 180000, 230000, 750000]
    })
    print("\nFiktívne dáta vytvorené.")
    print(data.head())


# 2. Predspracovanie textu (Text Preprocessing)
# Cieľom je "vyčistiť" text, aby bol použiteľný pre strojové učenie.
def predspracuj_text(text):
    text = text.lower()  # Všetko na malé písmená
    text = re.sub(r'\d+', '', text) # Odstráni čísla
    text = re.sub(r'[^\w\s]', '', text) # Odstráni interpunkciu

    # Odstránenie stopwords (časté slová ako 'a', 'je', 'na', ktoré nenesú veľký význam)
    # stop_words = set(stopwords.words('slovak')) # Pre slovenčinu, ak máš stiahnuté
    # Ak nemáš slovak stopwords, použijeme anglické, alebo žiadne pre zjednodušenie
    # Pre reálne použitie by si potreboval slovenské stopwords.
    stop_words = set(['a', 'je', 'na', 'v', 'sa', 'pre', 's', 'do', 'za', 'z', 'so', 'od', 'po', 'k', 'ku', 'aj', 'či', 'ale', 'no', 'že', 'lebo', 'teda'])

    # Stemming (zredukovanie slov na ich koreň) - napr. "domov", "domy" -> "dom"
    # Toto je často diskutabilné pre slovenčinu kvôli ohýbaniu. Pre jednoduchosť môžeme vynechať alebo použiť základný stemmer.
    ps = PorterStemmer() # Anglický stemmer, pre slovenčinu treba iný alebo lemmatizáciu

    words = text.split()
    # words = [ps.stem(word) for word in words if word not in stop_words] # S stemmingom a stopwords
    words = [word for word in words if word not in stop_words] # Len s stopwords

    return " ".join(words)

data['cisty_popis'] = data['popis_nehnutelnosti'].apply(predspracuj_text)
print("\nPredspracovaný text (prvých 5 riadkov):")
print(data[['popis_nehnutelnosti', 'cisty_popis']].head())

# 3. Vytvorenie číselnej reprezentácie textu (Feature Engineering - TF-IDF)
# Strojové učenie pracuje s číslami, nie s textom. TF-IDF prevádza text na vektory čísel.
# TF-IDF (Term Frequency-Inverse Document Frequency) dáva väčšiu váhu slovám,
# ktoré sú dôležité pre konkrétny dokument, ale zároveň nie sú príliš bežné v celom korpuse.
vectorizer = TfidfVectorizer(max_features=1000) # Obmedzíme počet najdôležitejších slov na 1000
X_text_features = vectorizer.fit_transform(data['cisty_popis'])

# Konvertovanie na DataFrame pre lepšiu manipuláciu (voliteľné)
X_text_df = pd.DataFrame(X_text_features.toarray(), columns=vectorizer.get_feature_names_out())
print(f"\nRozmery textových príznakov: {X_text_df.shape}")
print("Niekoľko ukážok TF-IDF príznakov:")
print(X_text_df.head())

# 4. Spojenie s ďalšími číselnými dátami (ak existujú, napr. počet izieb, plocha)
# Pre tento príklad nemáme ďalšie číselné dáta z inzerátu, ale v reálnom svete by ste ich pripojili.
# Napr.: data['pocet_izieb'] = [3, 1, 4, 2, 4, 3, 5]
#        X_all_features = pd.concat([X_text_df, data[['pocet_izieb']]], axis=1)
# Pre náš príklad použijeme len textové príznaky
X = X_text_df
y = data['cena']

# 5. Rozdelenie dát na trénovaciu a testovaciu sadu
# Trénovacia sada sa použije na "učenie" modelu, testovacia na vyhodnotenie jeho výkonu.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nRozmery trénovacej sady (X_train): {X_train.shape}")
print(f"Rozmery testovacej sady (X_test): {X_test.shape}")

# 6. Trénovanie modelu strojového učenia (Lineárna Regresia)
# Lineárna regresia je jednoduchý, ale silný model pre odhad číselných hodnôt.
model = LinearRegression()
model.fit(X_train, y_train)

print("\nModel Lineárnej Regresie bol natrénovaný.")

# 7. Vyhodnotenie modelu
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\nVýsledky vyhodnotenia modelu:")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} (nižšie je lepšie)")
print(f"R-squared (R2): {r2:.2f} (bližšie k 1 je lepšie)")

# 8. Predikcia novej nehnuteľnosti
nova_nehnutelnost_popis = "Priestranný byt s terasou v novostavbe, blízko jazera. Tiché prostredie."
cisty_novy_popis = predspracuj_text(nova_nehnutelnost_popis)

# Dôležité: Nový text musí prejsť rovnakým vektorizátorom, aký bol použitý na trénovacie dáta
nova_nehnutelnost_features = vectorizer.transform([cisty_novy_popis])
nova_nehnutelnost_features_df = pd.DataFrame(nova_nehnutelnost_features.toarray(), columns=vectorizer.get_feature_names_out())

# Zabezpečíme, že stĺpce sú v rovnakom poradí ako pri trénovaní (veľmi dôležité!)
# Toto je nutné, ak X_train bol pandas DataFrame. Ak bol X_text_features scipy sparse matrix, nie je to tak kritické.
# Pre istotu by sme mali zabezpečiť, že 'nova_nehnutelnost_features_df' má všetky stĺpce ako 'X_train'
# a chýbajúce vyplniť nulami, ak je to potrebné.
# Pre zjednodušený príklad to zatiaľ ignorujeme.

odhadovana_cena = model.predict(nova_nehnutelnost_features_df)[0]
print(f"\nOdhadovaná cena pre '{nova_nehnutelnost_popis}': {odhadovana_cena:.2f} EUR")

Súbor 'reality_data.csv' sa nenašiel. Prosím, uistite sa, že súbor existuje v rovnakom adresári.
Pre spustenie príkladu vytvoríme fiktívne dáta.

Fiktívne dáta vytvorené.
                                 popis_nehnutelnosti    cena
0  Priestranný 3-izbový byt v centre mesta, blízk...  250000
1  Malý 1-izbový byt na okraji mesta, potrebuje r...  120000
2  Veľký rodinný dom s garážou a záhradou na dedi...  350000
3  Moderný 2-izbový byt v novostavbe s výhľadom n...  280000
4  Starší 4-izbový dom v pôvodnom stave, veľký po...  180000

Predspracovaný text (prvých 5 riadkov):
                                 popis_nehnutelnosti  \
0  Priestranný 3-izbový byt v centre mesta, blízk...   
1  Malý 1-izbový byt na okraji mesta, potrebuje r...   
2  Veľký rodinný dom s garážou a záhradou na dedi...   
3  Moderný 2-izbový byt v novostavbe s výhľadom n...   
4  Starší 4-izbový dom v pôvodnom stave, veľký po...   

                                         cisty_popis  
0  priestranný izbový byt cent